## Mount
- 혹시 colab에서 돌려보게 되면 주석 풀고 ㄱㄱ

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [5]:
import pandas as pd
import numpy as np
# for문 진행상황 확인용
from tqdm import tqdm 
# 한국어 텍스트 분석 라이브러리. 참고로 vscode에서 깔려면 java.. 뭐시기 환경 설정해야해서 colab 고
from konlpy.tag import Okt 
#문자열 전처리 라이브러리
import re 
from collections import Counter 

## Data
- 경로가 colab 기준으로 되어있어서 각자 경로로 변경
- 데이터도 몇차 데이터임 ㅋㅋ 그것 또한 변경

In [6]:
train = pd.read_csv("/content/drive/MyDrive/강석호교수님.csv")
test = pd.read_csv("/content/drive/MyDrive/강명인교수님 (2).csv")

## Dictionary & Scoring Matrix
- X(STT 결과)와 True y(진짜 자막 단 것)을 비교해가면서 {잘못 발음된 단어:진짜 단어} 형태의 사전 구축
- Train 데이터에 나오는 모든 X 데이터의 단어 별로 얼마나 잘 맞췄는지를 scoring 해서 사전 구축
    - 규칙기반으로 가기 때문에 inference 데이터가 들어오면 규칙에 맞게 적용을 시켜줘야 하는데 모든 단어를 다 바꿔주기에는 시간이 오래 걸림
    - 특별한 단어(전공 단어)가 문제지 일상 언어에 대해서는 정확도가 매우 높으므로 굳이 바꿔줄 필요가 없음 -> 한 마디로 특정 주제에 대해 정확도를 높여주는 전문가를 만들어준다고 생각하자

In [7]:
total_word = [word for sentence in train['X'] for word in sentence.split()] # train X 데이터를 모두 띄어쓰기 단위로 쪼개서 list에 저장(중복 포함)
count_word = Counter(total_word) # {단어:데이터에서 해당 단어가 등장한 횟수}
unique_word = list(set([word for sentence in train['X'] for word in sentence.split()])) # list -> set -> list로 바꿔서 중복을 제거하고 list로 만듦

score = np.zeros(len(unique_word),) # [0,0,0, ..] 으로 이루어진 matrix인데 전체 단어 종류만큼 열이 존재
word_idx = {word:idx for idx, word in enumerate(unique_word)} # {단어:matrix에서의 위치}
original = np.array([count_word[word] for word in unique_word]) # 나중에 score을 [0,1] 사이를 갖게 하기 위해 단어의 총 등장 횟수를 matrix(score과 동일한 size)로 저장

In [8]:
# 발음대로 기록된 것과 실제는 다름 -> 구어를 바로 텍스트로 변환한 것의 특징상 띄어쓰기로 나눴을 때 X와 True y의 길이가 달라짐
lower_list = []
same_list = []
higher_list = []
for i in tqdm(range(len(train))):
    X_train = [word for word in train.loc[i,'X'].split()]
    y_train = [word for word in train.loc[i,'y'].split()]
    if len(X_train) > len(y_train): # ex) ['함', '문'] -> ['학문']
      lower_list.append(i)
    elif (X_train != y_train) and (len(X_train) == len(y_train)): # ex) ['뉴로이넷'] -> ['뉴럴네트워크']
      same_list.append(i)
    elif len(X_train) < len(y_train): # ex) ['변환', '한', '것'] -> ['변환한', '것']
      higher_list.append(i)
    elif X_train == y_train: 
      for word in X_train:
        score[word_idx[word]] += 1 # X와 True y가 일치하면 score 1점 줌

100%|██████████| 805/805 [00:00<00:00, 21136.81it/s]


In [9]:
# 케이스별로 나눈 데이터를 데이터프레임 형태로 바꾸고 index도 정렬
lower_df = train[train.index.isin(lower_list)].reset_index(drop=True)
same_df = train[train.index.isin(same_list)].reset_index(drop=True)
higher_df = train[train.index.isin(higher_list)].reset_index(drop=True)

In [230]:
key_list = []
value_list = []

# X와 True y가 일대일 대응
for idx in range(len(same_df)):
    arr1 = np.array([word for word in same_df.loc[idx, 'X'].split()])
    arr2 = np.array([word for word in same_df.loc[idx, 'y'].split()])
    wrong = arr1[arr1!=arr2] # 틀린 단어 
    right = arr1[arr1==arr2] # 맞는 단어
    correction = arr2[arr1!=arr2] # 틀린 단어의 원래 정답
    for i in range(len(wrong)):
      key_list.append(wrong[i]) # 틀린 단어는 추후 사전의 key
      value_list.append(correction[i]) # 원래 정답은 추후 사전의 value
    for k in range(len(right)):
      score[word_idx[right[k]]] += 1 # 맞은 단어는 score 1점씩 또 줌

correction_dict = dict(zip(key_list, value_list)) # {틀린 단어:원래 정답}으로 딕셔너리

In [231]:
# ['함', '문'] -> ['학문']과 같이 하나의 단어가 두 개 이상의 단어로 나눠져서 STT 출력됨
for idx in range(len(lower_df)):
    list1 = [word for word in lower_df.loc[idx,'X'].split()]
    list2 = [word for word in lower_df.loc[idx,'y'].split()]
    intersection = list(set(list1).intersection(set(list2))) # X와 True y의 교집합
    wrong = lower_df.loc[idx,'X'] # 우선 X 한 문장 가져옴
    correction = lower_df.loc[idx,'y'] # y도 한 문장 가져옴
    for i in intersection: # 교집합 단어(맞게 출력한 단어)에 대해서
        if i in wrong: 
            wrong = wrong.replace(i, "") # X 문장에 맞게 출력한 단어는 없애버림
        if i in correction:
            correction = correction.replace(i, "") # y 문장에 맞게 출력한 단어는 없애버림
        score[word_idx[i]] += 1 # 교집합은 맞게 출력한 단어니 또 1점씩 

    if len(list(filter(None, wrong.split("  ")))) == len(list(filter(None, correction.split("  ")))): # ["할 수"] -> ["함수"]와 같이 "할 수"가 하나로 묶였는지 확인
        for k in range(len(list(filter(None, wrong.split("  "))))):
            correction_dict[list(filter(None, wrong.split("  ")))[k]] = list(filter(None, correction.split("  ")))[k] # {"할 수":"함수"}로 correction_dict에 추가


In [232]:
# 위랑 똑같은 방식으로 함. 대신 ["함수"] -> ["할", "수"] 이런 데이터를 대상으로
for idx in range(len(higher_df)):
    list1 = [word for word in higher_df.loc[idx,'X'].split()]
    list2 = [word for word in higher_df.loc[idx,'y'].split()]
    intersection = list(set(list1).intersection(set(list2)))
    wrong = higher_df.loc[idx,'X']
    correction = higher_df.loc[idx,'y']
    for i in intersection:
        if i in wrong:
            wrong = wrong.replace(i, "")
        if i in correction:
            correction = correction.replace(i, "")
        score[word_idx[i]] += 1

    if len(list(filter(None, wrong.split("  ")))) == len(list(filter(None, correction.split("  ")))):
        for k in range(len(list(filter(None, wrong.split("  "))))):
            correction_dict[list(filter(None, wrong.split("  ")))[k]] = list(filter(None, correction.split("  ")))[k]

## 딕셔너리의 범용성을 높이기 위한 과정
- 현재 사전에는 "뉴럴렛", "뉴럴렛이", "뉴럴렛은" 이런 애들이 다 들어가 있어서 "뉴럴렛"으로 대표할 수 있게끔 만들어주는 것
- 최대한 기존의 구축되어 있는 Okt 한국어 라이브러리를 사용하고 싶었지만 영어랑 섞여있거나 그냥 잘 못해줄 때가 많음 -> 규칙기반으로 좀 더 조건은 걸었지만 그래도 완벽하지는 못함
    - !!!!!!!!!!!! 어쨌든 전처리 과정을 거쳐 최대한의 명사만 넣어놨는데 규칙기반으로 바꿔버리다 보니 inference 단계에서 조사를 다 빼먹어버림 ..
    - 우선은 안 돌렸다. 추가 실험을 해볼지도 ..

In [233]:
okt = Okt() # 한국어 라이브러리 불러오기 

In [235]:
new_key_list = []
new_value_list = []

for idx in range(len(correction_dict)):

  key = list(correction_dict.keys())[0] # key를 하나 뽑아오자
  value = correction_dict[key] # key에 맞는 value도 뽑음
  del correction_dict[key] # key와 value를 전처리해서 다시 넣을 것이기 때문에 기존의 key와 value는 제거

  # 영어, 한국어 제외 문자 제거
  key = re.sub('[^a-zA-Z가-힣]',' ',key) 
  value = re.sub('[^a-zA-Z가-힣]',' ',value)

  # 조사를 없애기 위한 노력 .. -> okt.morphs는 형태소 분석이라 ["자막", "이"]로 출력 + okt.nouns는 명사만 뽑아서 ["자막"] 출력 -> set로 만들어 차집합 해버리면 ("이")만 남음
  key_side = set(okt.morphs(key)) - set(okt.nouns(key))
  value_side = set(okt.morphs(value)) - set(okt.nouns(value))

  # {"리그레이션이":"리그레션이"} 이런식으로 조사가 발음에 영향을 미치지 않는 경우만 고려하기 위함(예외: {"논리니와":"논리니어"} -> "와"가 조사로 빠지지만 실제 조사로 쓰인게 아니라 발음상의 결과)
  if key_side == value_side:
    for i in range(len(key_side)):
      key = key.replace(list(key_side)[i], "") # key에서 조사 제거
      value = value.replace(list(value_side)[i], "") # value에서 조사 제거

  new_key_list.append(key) 
  new_value_list.append(value)

correction_dict = dict(zip(new_key_list, new_value_list)) # 전처리 완료된 key와 value를 다시 dictioinary로


In [190]:
del_list = []

# ["뉴럴렛", "뉴럴렛이", "뉴럴렛과"] -> ["뉴럴렛"] 위 과정을 거쳐도 조사가 여전히 붙어있기도 하고, list를 dictionary에 넣다보니 중복되기도 함
for key in list(correction_dict.keys()): # key를 하나 뽑자
  for other in list(set(correction_dict.keys())-set(key)): # 뽑았던 key를 제외하고 다른 key(other)를 뽑자
    if (other.find(key) != -1) and (key != other) and (key != " "): # other("뉴럴렛이")이 key("뉴럴렛")를 포함하고 있는지 확인
      del_list.append(other) # "뉴럴렛"이 있으므로 "뉴럴렛이"는 젝

In [191]:
for del_key in del_list:
  try:
    del correction_dict[del_key] # dictionary에서 조사가 붙은 놈들은 없애주세요
  except:
    continue

## Correction

In [192]:
normalize = score/original # X와 True y를 비교하면서 맞게 출력된 단어들은 점수를 줬음 -> 전체 등장 횟수로 나눠서 normalized score
# 참고로 normalized score을 histogram으로 찍어봤는데 한번씩 등장한 단어들이 많기도 하고, 일상어는 대체적으로 잘 맞춰서 score=1이 압도적으로 많음. 그 다음은 아예 0. 사잇값들도 있긴 한데 매우 적음
score_dict = dict(zip(unique_word, normalize)) # {단어:score}

In [270]:
from difflib import get_close_matches # 비슷한 문자열 찾는 파이썬 내장 모듈

candidate1 = list(score_dict.keys()) # 전체 train 데이터의 단어
candidate2 = list(correction_dict.keys()) # 수정이 필요할 단어 후보들

n=1 # 가장 비슷한 단어 하나만

# 비슷할 확률이 0.7 이상인 애들만(만약 모든 후보들이 0.7을 못 넘으면 빈 list 출력)
cutoff1 = 0.7
cutoff2 = 0.7

In [274]:
inference_list = []
for sentence in tqdm(test['X']): # Inference하고자 하는 데이터에서 문장 하나씩 가져오자
  tokenizer = sentence.split() # 띄어쓰기 단어로 쪼개서 list 만들자
  for token in tokenizer: # 문장의 단어 하나씩 가져오자
    try: 
      # 우리가 STT 정확도를 알고 있는 단어거나 알고 있는 단어 중에서 가장 비슷한 단어의 추정 정확도가 1이면 출력된 문장 그대로 가자
      if score_dict[get_close_matches(token, candidate1, n, cutoff1)[0]]==1: 
        sentence = sentence
      # 우리가 STT 정확도를 알고 있는 단어거나 알고 있는 단어 중에서 가장 비슷한 단어의 추정 정확도가 1 이하면 교정을 하자
      elif (score_dict[get_close_matches(token, candidate1, n, cutoff1)[0]]!=1):
        sentence = sentence.replace(token, correction_dict[get_close_matches(token, candidate1, n, cutoff1)[0]])

    except: # except의 이유는 위 elif에서 correctoin_dict의 key 위치로 들어오는 단어가 correction_dict에 없을 수도 있음 혹은 cutoff를 넘는 단어가 없어서 빈 list를 반환했거나
      try: # 그렇다면 correction_dict의 key 중에서 가장 유사한 애를 직접적으로 뽑아 대체하자(정확도가 떨어질 수 밖에)
        sentence = sentence.replace(token, correction_dict[get_close_matches(token, candidate2, n, cutoff2)[0]])
      except: # correction_dict의 key와 유사도에서도 cutoff를 넘긴 단어가 없으면 그냥 원본 그대로 쓰자
        sentence = sentence

  inference_list.append(sentence)

100%|██████████| 648/648 [00:20<00:00, 31.36it/s]


In [275]:
correction = pd.DataFrame(inference_list, columns=["correction"])

total = pd.concat([test['X'],test['y'],correction], axis=1) # X, True y, y hat

In [276]:
total

,X,y,correction
0,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분
1,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다
2,오늘 할 내용은 뉴로이 네트워크 이구요,오늘 할 내용은 뉴럴 네트워크 이구요,오늘 할 내용은 뉴로이 network 이구요
3,원래 서포트 벡터 머신이 먼저 나와 있는데,원래 서포트 벡터 머신이 먼저 나와 있는데,원래 서포트 벡터 머신이 먼저 나와 있는데
4,"순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다","순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다","순서를 바꿔서 인공신경망, 뉴로이 network 관련된 내용부터 진행하도록 하겠습니다"
...,...,...,...
643,조금 더 빠르게 수렴할 수 수렴시킬 수 있고,조금 더 빠르게 수렴할 수 수렴시킬 수 있고,조금 더 빠르게 수렴할 수 수렴시킬 수 있고
644,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요
645,그런데 이제 사실 만약에 사이클론 말고,그런데 이제 사실 만약에 사이킷런 말고,그런데 이제 사실 만약에 사이킷런 말고
646,여러분이 뭔가 좀 더 텐서플로우나,여러분이 뭔가 좀 더 텐서플로우나,여러분이 뭔가 좀 더 텐서플로우나
